In [6]:
import pandas as pd
import os
import re
from utils import _strip_string,delete_extra_zero

In [27]:
MODEL_NAME = 'GPT4'
CATEGORY = 'Math'
DF_NAME = 'GSM8K'
dir_pth = f'../data/ES_data/{MODEL_NAME}_result'
df_t07 = pd.read_json(os.path.join(dir_pth,'gsm8k/T0.7.jsonl'), lines=True)

In [10]:
df_t07

,question,answer,generated_answer
0,Q:Mico and Marco wanted to get to know each ot...,There will be an additional 10 (for Mico) + 10...,"[A: Currently, the sum of their ages is 20. In..."
1,Q:Hannah slips on a banana peel and breaks her...,First find the length of the visit in hours: 3...,[A: The cast costs $200. The doctor charges $3...
2,"Q:Judy teaches 5 dance classes, every day, on ...",She teaches 5 dance classes 5 days a week so t...,[A: Judy teaches 5 classes a day from Monday t...
3,Q:Nani is 8 years old. His brother is twice hi...,Nani's brother is 8 * 2 = <<8*2=16>>16 years o...,[A: Nani is 8 years old. His brother is twice ...
4,Q:Tobias bought a big pizza with 60 pieces. He...,"On the first day, Tobias ate 2/5*60 = <<2/5*60...","[A: On the first day, Tobias ate 2/5 of the pi..."
...,...,...,...
1314,Q:At the trip to the county-level scavenger hu...,Since the total number of people at the compet...,[A: The number of groups formed is 90 divided ...
1315,Q:Mandy owes Benedict $100. They agreed to hav...,The monthly interest is $100 x 2/100 = $<<100*...,[A: The interest for one month is 2% of 100 do...
1316,Q:Greta and Celinda are baking cookies. Greta ...,Celinda bakes 30 cookies * 2 = <<30*2=60>>60 c...,[A: Greta baked 30 cookies and Celinda baked t...
1317,Q:Jeff is 10 years older than his younger sist...,Martha is 24 - 4 = <<24-4=20>>20 years old.\nJ...,[A: If Mike is 24 years old and Martha is 4 ye...


In [7]:
ANS_RE = re.compile(r"#### (\-?[0-9\.\,]+)")
INVALID_ANS = "[invalid]"


def extract_answer(completion):
    match = ANS_RE.search(completion)
    if match:
        match_str = match.group(1).strip()
        match_str = match_str.replace(",", "")
        return match_str
    else:
        return INVALID_ANS


def extract_math_answer(pred_str):
    if ('The answer is ' in pred_str):
        pred = pred_str.split('The answer is ')[-1].strip()
    elif ('the answer is ' in pred_str):
        pred = pred_str.split('the answer is ')[-1].strip()
    elif 'boxed' in pred_str:
        ans = pred_str.split('boxed')[-1]
        if (ans[0] == '{'):
            stack = 1
            a = ''
            for c in ans[1:]:
                if (c == '{'):
                    stack += 1
                    a += c
                elif (c == '}'):
                    stack -= 1
                    if (stack == 0): break
                    a += c
                else:
                    a += c
        else:
            a = ans.split('$')[0].strip()
        a = _strip_string(a)
        pred = a

    else:
        pattern = '-?\d*\.?\d+'
        pred = re.findall(pattern, pred_str)
        if (len(pred) >= 1):
            # print(pred_str)
            pred = pred[-1]
        else:
            pred = ''
    if pred != "":
        if pred[-1] == ".":
            pred = pred[:-1]
        if pred[-1] == "/":
            pred = pred[:-1]
    pred = _strip_string(pred)
    if 'boxed' in pred:
        ans = pred.split('boxed')[-1]
        if (ans[0] == '{'):
            stack = 1
            a = ''
            for c in ans[1:]:
                if (c == '{'):
                    stack += 1
                    a += c
                elif (c == '}'):
                    stack -= 1
                    if (stack == 0): break
                    a += c
                else:
                    a += c
        else:
            a = ans.split('$')[0].strip()
        a = _strip_string(a)
        pred = a
    return pred

In [28]:
def save_csv(row):
    storage_dir = os.path.join('../data', f'ES_data/{MODEL_NAME}_result/{DF_NAME}')
    if not os.path.exists(storage_dir):
        os.makedirs(storage_dir)
    file_path = os.path.join(storage_dir, f'{DF_NAME}_cleaned.csv')

    if not os.path.isfile(file_path):
        # The file does not exist, write with header
        row.to_frame().T.to_csv(file_path, mode='a', index=False, header=True)
    else:
        # The file exists, append without header
        row.to_frame().T.to_csv(file_path, mode='a', index=False, header=False)

In [33]:
save_dict = {
    'Category': [CATEGORY for i in range(len(df_t07))],
    'Question':[None for i in range(len(df_t07))],
    'Correct Answer': [None for i in range(len(df_t07))],
}
df_save = pd.DataFrame(save_dict)
for row_idx in range(len(df_t07)):
    row = df_t07.iloc[row_idx]
    row_save = df_save.iloc[row_idx]
    row_save['Question']= row['question']
    right_answer = delete_extra_zero(extract_answer(row['answer']))
    row_save['Correct Answer'] = right_answer
    cot_li = row['generated_answer']
    for idx,cot in enumerate(cot_li):
        tem_1 = cot
        number_list = re.findall(r"\d+\.?\d*", tem_1)
        try:
            predict1 = number_list[-1].strip('.')
        except:
            predict1 = -1000
            # continue
        row_save[f'CoT_{idx}'] = cot
        row_save[f'Final Answer_{idx}'] = predict1
        row_save[f'Instruction Violation_{idx}'] = [(0,0)]
    save_csv(row_save)

In [36]:
import numpy as np
np.array([1,2,4]) == [4,2,6]

array([False,  True, False])